In [1]:
import os
import sys
import torch
import argparse
import torchvision
import numpy as np
import torch.nn as nn
import scipy.io as sio
from PIL import Image
from tqdm import tqdm
from datetime import datetime
from collections import defaultdict
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from os.path import join, isdir, abspath, dirname
import pywt
import pywt.data
# Customized import.
from networks import HED
from datasets import BsdsDataset
from utils import Logger, AverageMeter, \
    load_checkpoint, save_checkpoint, load_vgg16_caffe, load_pretrained_caffe


# # Parse arguments.
# parser = argparse.ArgumentParser(description='HED training.')
# # 1. Actions.
# parser.add_argument('--test',             default=False,             help='Only test the model.', action='store_true')
# # 2. Counts.
# parser.add_argument('--train_batch_size', default=1,    type=int,   metavar='N', help='Training batch size.')
# parser.add_argument('--test_batch_size',  default=1,    type=int,   metavar='N', help='Test batch size.')
# parser.add_argument('--train_iter_size',  default=10,   type=int,   metavar='N', help='Training iteration size.')
# parser.add_argument('--max_epoch',        default=40,   type=int,   metavar='N', help='Total epochs.')
# parser.add_argument('--print_freq',       default=500,  type=int,   metavar='N', help='Print frequency.')
# # 3. Optimizer settings.
# parser.add_argument('--lr',               default=1e-6, type=float, metavar='F', help='Initial learning rate.')
# parser.add_argument('--lr_stepsize',      default=1e4,  type=int,   metavar='N', help='Learning rate step size.')
# # Note: Step size is based on number of iterations, not number of batches.
# #   https://github.com/s9xie/hed/blob/94fb22f10cbfec8d84fbc0642b224022014b6bd6/src/caffe/solver.cpp#L498
# parser.add_argument('--lr_gamma',         default=0.1,  type=float, metavar='F', help='Learning rate decay (gamma).')
# parser.add_argument('--momentum',         default=0.9,  type=float, metavar='F', help='Momentum.')
# parser.add_argument('--weight_decay',     default=2e-4, type=float, metavar='F', help='Weight decay.')
# # 4. Files and folders.
# parser.add_argument('--vgg16_caffe',      default='',                help='Resume VGG-16 Caffe parameters.')
# parser.add_argument('--checkpoint',       default='',                help='Resume the checkpoint.')
# parser.add_argument('--caffe_model',      default='',                help='Resume HED Caffe model.')
# parser.add_argument('--output',           default='./output_LL_bior',        help='Output folder.')
# parser.add_argument('--dataset',          default='./data/HED-BSDS/LL_bior', help='HED-BSDS dataset folder.')
# # 5. Others.
# parser.add_argument('--cpu',              default=False,             help='Enable CPU mode.', action='store_true')
# args = parser.parse_args()

# Set device.
device = torch.device('cuda')

print(device)

def train(train_loader, net, opt, lr_schd, epoch, save_dir):
    """ Training procedure. """
    # Create the directory.
    if not isdir(save_dir):
        os.makedirs(save_dir)
    # Switch to train mode and clear the gradient.
    net.train()
    opt.zero_grad()
    # Initialize meter and list.
    batch_loss_meter = AverageMeter()
    # Note: The counter is used here to record number of batches in current training iteration has been processed.
    #       It aims to have large training iteration number even if GPU memory is not enough. However, such trick
    #       can be used because batch normalization is not used in the network architecture.
    counter = 0
    for batch_index, (images, edges) in enumerate(tqdm(train_loader)):
        # Adjust learning rate and modify counter following Caffe's way.
        if counter == 0:
            lr_schd.step()  # Step at the beginning of the iteration.
        counter += 1
        
        # take DWT of image here and set edges to LL
        # edges is in pytorch format
        
        edges = wave_trans(edges)
        
        
        # Get images and edges from current batch.
        images, edges = images.to(device), edges.to(device)
        # Generate predictions.
        preds_list = net(images)
        # Calculate the loss of current batch (sum of all scales and fused).
        # Note: Here we mimic the "iteration" in official repository: iter_size batches will be considered together
        #       to perform one gradient update. To achieve the goal, we calculate the equivalent iteration loss
        #       eqv_iter_loss of current batch and generate the gradient. Then, instead of updating the weights,
        #       we continue to calculate eqv_iter_loss and add the newly generated gradient to current gradient.
        #       After iter_size batches, we will update the weights using the accumulated gradients and then zero
        #       the gradients.
        # Reference:
        #   https://github.com/s9xie/hed/blob/94fb22f10cbfec8d84fbc0642b224022014b6bd6/src/caffe/solver.cpp#L230
        #   https://www.zhihu.com/question/37270367
        
        
        batch_loss = sum([weighted_cross_entropy_loss(preds, edges) for preds in preds_list])
        eqv_iter_loss = batch_loss / train_iter_size

        # Generate the gradient and accumulate (using equivalent average loss).
        eqv_iter_loss.backward()
        if counter == train_iter_size:
            opt.step()
            opt.zero_grad()
            counter = 0  # Reset the counter.
        # Record loss.
        batch_loss_meter.update(batch_loss.item())
        # Log and save intermediate images.
        if batch_index % print_freq == print_freq - 1:
            # Log.
            print(('Training epoch:{}/{}, batch:{}/{} current iteration:{}, ' +
                   'current batch batch_loss:{}, epoch average batch_loss:{}, learning rate list:{}.').format(
                   epoch, max_epoch, batch_index, len(train_loader), lr_schd.last_epoch,
                   batch_loss_meter.val, batch_loss_meter.avg, lr_schd.get_lr()))
            # Generate intermediate images.
            preds_list_and_edges = preds_list + [edges]
            _, _, h, w = preds_list_and_edges[0].shape
            interm_images = torch.zeros((len(preds_list_and_edges), 1, h, w))
            for i in range(len(preds_list_and_edges)):
                # Only fetch the first image in the batch.
                interm_images[i, 0, :, :] = preds_list_and_edges[i][0, 0, :, :]
            # Save the images.
            torchvision.utils.save_image(interm_images, join(save_dir, 'batch-{}-1st-image.png'.format(batch_index)))
    # Return the epoch average batch_loss.
    return batch_loss_meter.avg


def test(test_loader, net, save_dir):
    """ Test procedure. """
    # Create the directories.
    if not isdir(save_dir):
        os.makedirs(save_dir)
    save_png_dir = join(save_dir, 'png')
    if not isdir(save_png_dir):
        os.makedirs(save_png_dir)
    save_mat_dir = join(save_dir, 'mat')
    if not isdir(save_mat_dir):
        os.makedirs(save_mat_dir)
    # Switch to evaluation mode.
    net.eval()
    # Generate predictions and save.
    assert test_batch_size == 1  # Currently only support test batch size 1.
    for batch_index, images in enumerate(tqdm(test_loader)):
        images = images.cuda()
        _, _, h, w = images.shape
        preds_list = net(images)
        fuse       = preds_list[-1].detach().cpu().numpy()[0, 0]  # Shape: [h, w].
        name       = test_loader.dataset.images_name[batch_index]
        sio.savemat(join(save_mat_dir, '{}.mat'.format(name)), {'result': fuse})
        Image.fromarray((fuse * 255).astype(np.uint8)).save(join(save_png_dir, '{}.png'.format(name)))
        # print('Test batch {}/{}.'.format(batch_index + 1, len(test_loader)))


def weighted_cross_entropy_loss(preds, edges):
    """ Calculate sum of weighted cross entropy loss. """
    # Reference:
    #   hed/src/caffe/layers/sigmoid_cross_entropy_loss_layer.cpp
    #   https://github.com/s9xie/hed/issues/7
    
        
    
    
    mask = (edges > 0.5).float()
    b, c, h, w = mask.shape
    num_pos = torch.sum(mask, dim=[1, 2, 3]).float()  # Shape: [b,].
    num_neg = c * h * w - num_pos                     # Shape: [b,].
    weight = torch.zeros_like(mask)
    weight[edges > 0.5]  = num_neg / (num_pos + num_neg)
    weight[edges <= 0.5] = num_pos / (num_pos + num_neg)
    # Calculate loss.
    losses = torch.nn.functional.binary_cross_entropy(preds.float(), edges.float(), weight=weight, reduction='none')
    loss   = torch.sum(losses) / b
    return loss




cuda


In [2]:
op_dir = 'output_HH_bior'
data_dir = './data/HED-BSDS/HH_bior'
lr = 1e-6
weight_decay = 2e-4
momentum = 0.9
lr_stepsize = 1e4
lr_gamma = 0.1
max_epoch = 5
test_batch_size = 1
train_iter_size = 10
print_freq = 500

In [3]:
def wave_trans(images):
    # convert to numpy
    images = images.numpy() # making images an nchw numpy array
    images = images[0,0,:,:] # making images an hw numpy array
    coeffs2 = pywt.dwt2(images, 'bior4.4')
    LL, (LH, HL, HH) = coeffs2

    # HH is an hw numpy array

    # convert HH into a torch tensor of NCHW

    height = HH.shape[0]
    width = HH.shape[1]
    HH = torch.from_numpy(HH)
    HH = torch.reshape(HH,(1, 1, height, width))
    
    return(HH)

In [4]:
################################################
# I. Miscellaneous.
################################################
# Create the output directory.
# current_dir = abspath(dirname(__file__))
current_dir = os.getcwd()
output_dir = join(current_dir, op_dir)



In [5]:
if not isdir(output_dir):
    os.makedirs(output_dir)

In [6]:

# Set logger.
now_str = datetime.now().strftime('%y%m%d-%H%M%S')
log = Logger(join(output_dir, 'log-{}.txt'.format(now_str)))
sys.stdout = log  # Overwrite the standard output.


module.conv1_1.weight      lr:    1 decay:1
module.conv1_1.bias        lr:    2 decay:0
module.conv1_2.weight      lr:    1 decay:1
module.conv1_2.bias        lr:    2 decay:0
module.conv2_1.weight      lr:    1 decay:1
module.conv2_1.bias        lr:    2 decay:0
module.conv2_2.weight      lr:    1 decay:1
module.conv2_2.bias        lr:    2 decay:0
module.conv3_1.weight      lr:    1 decay:1
module.conv3_1.bias        lr:    2 decay:0
module.conv3_2.weight      lr:    1 decay:1
module.conv3_2.bias        lr:    2 decay:0
module.conv3_3.weight      lr:    1 decay:1
module.conv3_3.bias        lr:    2 decay:0
module.conv4_1.weight      lr:    1 decay:1
module.conv4_1.bias        lr:    2 decay:0
module.conv4_2.weight      lr:    1 decay:1
module.conv4_2.bias        lr:    2 decay:0
module.conv4_3.weight      lr:    1 decay:1
module.conv4_3.bias        lr:    2 decay:0
module.conv5_1.weight      lr:  100 decay:1
module.conv5_1.bias        lr:  200 decay:0
module.conv5_2.weight      lr:  

Training epoch:0/5, batch:11499/28800 current iteration:1150, current batch batch_loss:106.50659942626953, epoch average batch_loss:339.0062308003384, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:0/5, batch:11999/28800 current iteration:1200, current batch batch_loss:485.8105163574219, epoch average batch_loss:339.1081452960173, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:0/5, batch:12499/28800 current iteration:1250, current batch batch_loss:311.0423889160156, epoch average batch_loss:338.44774646751404, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:0/5, batch:12999/28800 current iteration:1300, current batch batch_loss:734.085693359375, epoch average batch_loss:337.9044689285205, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.0001999999

Training epoch:0/5, batch:27499/28800 current iteration:2750, current batch batch_loss:305.0498046875, epoch average batch_loss:334.55234051478993, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:0/5, batch:27999/28800 current iteration:2800, current batch batch_loss:136.37916564941406, epoch average batch_loss:334.37582981470655, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:0/5, batch:28499/28800 current iteration:2850, current batch batch_loss:163.92626953125, epoch average batch_loss:334.23208864078185, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:1/5, batch:499/28800 current iteration:2930, current batch batch_loss:124.10095977783203, epoch average batch_loss:328.1586467933655, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.0001999999999

Training epoch:1/5, batch:14999/28800 current iteration:4380, current batch batch_loss:450.17138671875, epoch average batch_loss:332.0891965379715, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:1/5, batch:15499/28800 current iteration:4430, current batch batch_loss:454.4606628417969, epoch average batch_loss:332.5321783746904, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:1/5, batch:15999/28800 current iteration:4480, current batch batch_loss:852.6973876953125, epoch average batch_loss:332.3656380594671, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:1/5, batch:16499/28800 current iteration:4530, current batch batch_loss:165.859375, epoch average batch_loss:332.1563144482988, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998

Training epoch:2/5, batch:2499/28800 current iteration:6010, current batch batch_loss:290.91546630859375, epoch average batch_loss:333.6817847545624, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:2/5, batch:2999/28800 current iteration:6060, current batch batch_loss:141.51348876953125, epoch average batch_loss:334.7920497090022, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:2/5, batch:3499/28800 current iteration:6110, current batch batch_loss:352.336669921875, epoch average batch_loss:333.6367403703417, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:2/5, batch:3999/28800 current iteration:6160, current batch batch_loss:504.7548828125, epoch average batch_loss:332.09663970541953, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.0001999999999999

Training epoch:2/5, batch:18499/28800 current iteration:7610, current batch batch_loss:100.64486694335938, epoch average batch_loss:327.26801728088793, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:2/5, batch:18999/28800 current iteration:7660, current batch batch_loss:306.197021484375, epoch average batch_loss:327.4942518464139, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:2/5, batch:19499/28800 current iteration:7710, current batch batch_loss:135.8361053466797, epoch average batch_loss:327.61653067045944, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:2/5, batch:19999/28800 current iteration:7760, current batch batch_loss:959.4257202148438, epoch average batch_loss:328.13084041371343, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999

Training epoch:3/5, batch:5999/28800 current iteration:9240, current batch batch_loss:266.31591796875, epoch average batch_loss:330.8312017457485, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:3/5, batch:6499/28800 current iteration:9290, current batch batch_loss:353.0539855957031, epoch average batch_loss:329.5439841350409, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:3/5, batch:6999/28800 current iteration:9340, current batch batch_loss:183.0209197998047, epoch average batch_loss:329.04464305680136, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.00019999999999999998, 1e-08, 2e-08, 1e-09, 2e-09].
Training epoch:3/5, batch:7499/28800 current iteration:9390, current batch batch_loss:57.01019287109375, epoch average batch_loss:329.5137782424291, learning rate list:[1e-06, 2e-06, 9.999999999999999e-05, 0.0001999999999999

Training epoch:3/5, batch:20999/28800 current iteration:10740, current batch batch_loss:364.5345764160156, epoch average batch_loss:328.06918877183824, learning rate list:[1e-07, 2e-07, 9.999999999999999e-06, 1.9999999999999998e-05, 1e-09, 2e-09, 1.0000000000000002e-10, 2.0000000000000003e-10].
Training epoch:3/5, batch:21499/28800 current iteration:10790, current batch batch_loss:409.2449951171875, epoch average batch_loss:328.2067622862084, learning rate list:[1e-07, 2e-07, 9.999999999999999e-06, 1.9999999999999998e-05, 1e-09, 2e-09, 1.0000000000000002e-10, 2.0000000000000003e-10].
Training epoch:3/5, batch:21999/28800 current iteration:10840, current batch batch_loss:237.63226318359375, epoch average batch_loss:328.424299477924, learning rate list:[1e-07, 2e-07, 9.999999999999999e-06, 1.9999999999999998e-05, 1e-09, 2e-09, 1.0000000000000002e-10, 2.0000000000000003e-10].
Training epoch:3/5, batch:22499/28800 current iteration:10890, current batch batch_loss:397.8233642578125, epoch a

In [7]:

################################################
# II. Datasets.
################################################
# Datasets and dataloaders.
train_dataset = BsdsDataset(dataset_dir=data_dir, split='train')
test_dataset  = BsdsDataset(dataset_dir=data_dir, split='test')
train_loader  = DataLoader(train_dataset, batch_size=1,
                           num_workers=4, drop_last=True, shuffle=True)
test_loader   = DataLoader(test_dataset,  batch_size=1,
                           num_workers=4, drop_last=False, shuffle=False)


In [8]:

################################################
# III. Network and optimizer.
################################################
# Create the network in GPU.
net = nn.DataParallel(HED(device))
net.to(device)

# Initialize the weights for HED model.
def weights_init(m):
    """ Weight initialization function. """
    if isinstance(m, nn.Conv2d):
        # Initialize: m.weight.
        if m.weight.data.shape == torch.Size([1, 5, 1, 1]):
            # Constant initialization for fusion layer in HED network.
            torch.nn.init.constant_(m.weight, 0.2)
        else:
            # Zero initialization following official repository.
            # Reference: hed/docs/tutorial/layers.md
            m.weight.data.zero_()
        # Initialize: m.bias.
        if m.bias is not None:
            # Zero initialization.
            m.bias.data.zero_()
net.apply(weights_init)

# Optimizer settings.
net_parameters_id = defaultdict(list)
for name, param in net.named_parameters():
    if name in ['module.conv1_1.weight', 'module.conv1_2.weight',
                'module.conv2_1.weight', 'module.conv2_2.weight',
                'module.conv3_1.weight', 'module.conv3_2.weight', 'module.conv3_3.weight',
                'module.conv4_1.weight', 'module.conv4_2.weight', 'module.conv4_3.weight']:
        print('{:26} lr:    1 decay:1'.format(name)); net_parameters_id['conv1-4.weight'].append(param)
    elif name in ['module.conv1_1.bias', 'module.conv1_2.bias',
                  'module.conv2_1.bias', 'module.conv2_2.bias',
                  'module.conv3_1.bias', 'module.conv3_2.bias', 'module.conv3_3.bias',
                  'module.conv4_1.bias', 'module.conv4_2.bias', 'module.conv4_3.bias']:
        print('{:26} lr:    2 decay:0'.format(name)); net_parameters_id['conv1-4.bias'].append(param)
    elif name in ['module.conv5_1.weight', 'module.conv5_2.weight', 'module.conv5_3.weight']:
        print('{:26} lr:  100 decay:1'.format(name)); net_parameters_id['conv5.weight'].append(param)
    elif name in ['module.conv5_1.bias', 'module.conv5_2.bias', 'module.conv5_3.bias'] :
        print('{:26} lr:  200 decay:0'.format(name)); net_parameters_id['conv5.bias'].append(param)
    elif name in ['module.score_dsn1.weight', 'module.score_dsn2.weight',
                  'module.score_dsn3.weight', 'module.score_dsn4.weight', 'module.score_dsn5.weight']:
        print('{:26} lr: 0.01 decay:1'.format(name)); net_parameters_id['score_dsn_1-5.weight'].append(param)
    elif name in ['module.score_dsn1.bias', 'module.score_dsn2.bias',
                  'module.score_dsn3.bias', 'module.score_dsn4.bias', 'module.score_dsn5.bias']:
        print('{:26} lr: 0.02 decay:0'.format(name)); net_parameters_id['score_dsn_1-5.bias'].append(param)
    elif name in ['module.score_final.weight']:
        print('{:26} lr:0.001 decay:1'.format(name)); net_parameters_id['score_final.weight'].append(param)
    elif name in ['module.score_final.bias']:
        print('{:26} lr:0.002 decay:0'.format(name)); net_parameters_id['score_final.bias'].append(param)

# Create optimizer.
opt = torch.optim.SGD([
    {'params': net_parameters_id['conv1-4.weight']      , 'lr': lr*1    , 'weight_decay': weight_decay},
    {'params': net_parameters_id['conv1-4.bias']        , 'lr': lr*2    , 'weight_decay': 0.},
    {'params': net_parameters_id['conv5.weight']        , 'lr': lr*100  , 'weight_decay': weight_decay},
    {'params': net_parameters_id['conv5.bias']          , 'lr': lr*200  , 'weight_decay': 0.},
    {'params': net_parameters_id['score_dsn_1-5.weight'], 'lr': lr*0.01 , 'weight_decay': weight_decay},
    {'params': net_parameters_id['score_dsn_1-5.bias']  , 'lr': lr*0.02 , 'weight_decay': 0.},
    {'params': net_parameters_id['score_final.weight']  , 'lr': lr*0.001, 'weight_decay': weight_decay},
    {'params': net_parameters_id['score_final.bias']    , 'lr': lr*0.002, 'weight_decay': 0.},
], lr=lr, momentum=momentum, weight_decay=weight_decay)
# Note: In train_val.prototxt and deploy.prototxt, the learning rates of score_final.weight/bias are different.

# Learning rate scheduler.
lr_schd = lr_scheduler.StepLR(opt, step_size=lr_stepsize, gamma=lr_gamma)


In [9]:

################################################
# IV. Pre-trained parameters.
################################################
# Load parameters from pre-trained VGG-16 Caffe model.
# if args.vgg16_caffe:
load_vgg16_caffe(net, './data/5stage-vgg.py36pickle')

# # Resume the checkpoint.
# if args.checkpoint:
#     load_checkpoint(net, opt, args.checkpoint)  # Omit the returned values.

# # Resume the HED Caffe model.
# if args.caffe_model:
#     load_pretrained_caffe(net, args.caffe_model)


In [ ]:
################################################
# V. Training / testing.
################################################
# if args.test is True:
#     # Only test.
#     test(test_loader, net, save_dir=join(output_dir, 'test'))
# else:
# Train.
train_epoch_losses = []
for epoch in range(max_epoch):
    # Initial test.
    if epoch == 0:
        print('Initial test...')
        test(test_loader, net, save_dir=join(output_dir, 'initial-test'))
    # Epoch training and test.
    train_epoch_loss = \
        train(train_loader, net, opt, lr_schd, epoch, save_dir=join(output_dir, 'epoch-{}-train'.format(epoch)))
    test(test_loader, net, save_dir=join(output_dir, 'epoch-{}-test'.format(epoch)))
    # Write log.
    log.flush()
    # Save checkpoint.
    save_checkpoint(state={'net': net.state_dict(), 'opt': opt.state_dict(), 'epoch': epoch},
                    path=os.path.join(output_dir, 'epoch-{}-checkpoint.pt'.format(epoch)))
    # Collect losses.
    train_epoch_losses.append(train_epoch_loss)


  0%|          | 0/28800 [00:00<?, ?it/s]/datasets/home/24/524/cpatil/.local/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
 84%|████████▎ | 24068/28800 [11:52<02:28, 31.76it/s]